In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='buy', pruner_type='sucessive',
        train_start=datetime(2020,2,1), train_end=datetime(2025,1,1),
        test_start=datetime(2022,1,1),  test_end=datetime(2023,6,1),
        label_method='filter', search_type='clusters', search_subtype='kmeans',
        n_models=1, debug=False,
    ),
]

# Actualizar n_jobs en cada configuración
for cfg in configs:
    cfg["tag"] = f"{cfg['symbol']}_{cfg['timeframe']}_{cfg['direction']}_{cfg['label_method']}_{cfg['search_type']}_{cfg.get('search_subtype', '')}".strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 500
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 39/538 score=0.634678 avg=1.37s mem=502.14MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 40/538 score=0.634678 avg=3.82s mem=502.90MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 41/538 score=0.634678 avg=4.69s mem=534.59MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 42/538 score=0.634678 avg=5.11s mem=510.42MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 43/538 score=0.634678 avg=6.53s mem=504.99MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 44/538 score=0.634678 avg=7.60s mem=547.64MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 45/538 score=0.634678 avg=9.09s mem=478.68MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 46/538 score=0.634678 avg=9.47s mem=498.34MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 47/538 score=0.634678 avg=9.84s mem=494.38MB 
[XAUUSD_H1_buy_filter_clusters_kmeans] modelo 0 trial 48/538 score=0.634678 avg=10.14s mem=517.48MB 